# Loop 14 Analysis: Strategic Assessment

## Key Observations
1. After 15 experiments, best score is 70.630478 (from saspav_best ensemble)
2. Target: 68.919154
3. Gap: 1.711 points (2.42%)
4. ALL approaches have failed to improve beyond 70.630478

## Critical Question: Is the target achievable?
Let's analyze what would need to happen to close the 1.711 point gap.

In [ ]:
import numpy as np
import pandas as pd

# Load current best
df = pd.read_csv('/home/code/exploration/datasets/saspav_best.csv')

# Calculate per-N scores
def calculate_per_n_scores(df):
    scores = {}
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    tx=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2])
    ty=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1])
    
    for n in range(1, 201):
        group = df[df['id'].str.startswith(f'{n:03d}_')]
        if len(group) == 0:
            continue
        xs = group['x'].str.lstrip('s').astype(float).values
        ys = group['y'].str.lstrip('s').astype(float).values
        degs = group['deg'].str.lstrip('s').astype(float).values
        
        mnx=1e300; mny=1e300; mxx=-1e300; mxy=-1e300
        for i in range(len(xs)):
            r = degs[i] * np.pi / 180.0
            c, s = np.cos(r), np.sin(r)
            for j in range(len(tx)):
                X = c*tx[j] - s*ty[j] + xs[i]
                Y = s*tx[j] + c*ty[j] + ys[i]
                mnx = min(mnx, X)
                mxx = max(mxx, X)
                mny = min(mny, Y)
                mxy = max(mxy, Y)
        side = max(mxx-mnx, mxy-mny)
        scores[n] = side*side / n
    return scores

scores = calculate_per_n_scores(df)
total = sum(scores.values())
print(f'Total score: {total:.6f}')
print(f'Target: 68.919154')
print(f'Gap: {total - 68.919154:.6f} ({(total - 68.919154) / 68.919154 * 100:.2f}%)')

In [ ]:
# Analyze score distribution by N ranges
ranges = [
    (1, 20, 'Small N (1-20)'),
    (21, 50, 'Medium N (21-50)'),
    (51, 100, 'Large N (51-100)'),
    (101, 200, 'Very Large N (101-200)')
]

print('Score breakdown by N range:')
print('='*60)
for start, end, name in ranges:
    range_score = sum(scores[n] for n in range(start, end+1))
    pct = range_score / total * 100
    print(f'{name}: {range_score:.4f} ({pct:.1f}%)')
    
    # What improvement needed to close gap from this range alone?
    gap = total - 68.919154
    needed_improvement = gap / range_score * 100
    print(f'  -> To close gap from this range alone: {needed_improvement:.1f}% improvement needed')

In [ ]:
# Find the N values with highest scores (most room for improvement)
print('\nTop 20 N values by score contribution:')
print('='*60)
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
for n, score in sorted_scores[:20]:
    print(f'N={n:3d}: {score:.6f}')

In [ ]:
# Calculate theoretical lower bound (area-based)
# Tree area = 0.7*0.25/2 + 0.4*0.25/2 + 0.25*0.3/2 + 0.15*0.2 = 0.1675 (approximate)
# Actually let's compute it properly

from shapely.geometry import Polygon

# Tree polygon
tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
coords = [
    (0, tip),
    (ow/2, t1), (ow/4, t1),
    (mw/2, t2), (mw/4, t2),
    (bw/2, base),
    (tw/2, base), (tw/2, tbot),
    (-tw/2, tbot), (-tw/2, base),
    (-bw/2, base),
    (-mw/4, t2), (-mw/2, t2),
    (-ow/4, t1), (-ow/2, t1)
]
tree = Polygon(coords)
tree_area = tree.area
print(f'Tree area: {tree_area:.6f}')

# Theoretical lower bound for N trees
print('\nTheoretical lower bounds (area-based):')
print('='*60)
theoretical_total = 0
for n in range(1, 201):
    # Minimum side = sqrt(n * tree_area)
    min_side = np.sqrt(n * tree_area)
    min_score = min_side**2 / n  # = tree_area (constant!)
    theoretical_total += min_score
    
print(f'Theoretical minimum (perfect packing): {theoretical_total:.6f}')
print(f'Current best: {total:.6f}')
print(f'Efficiency: {theoretical_total / total * 100:.2f}%')

In [ ]:
# The theoretical minimum is based on perfect packing (no wasted space)
# In reality, irregular polygons can't pack perfectly
# Let's estimate realistic efficiency

# For irregular polygons, typical packing efficiency is 60-80%
# This means side length is sqrt(n * tree_area / efficiency)

print('Realistic bounds based on packing efficiency:')
print('='*60)
for efficiency in [0.5, 0.6, 0.7, 0.8]:
    realistic_total = 0
    for n in range(1, 201):
        min_side = np.sqrt(n * tree_area / efficiency)
        min_score = min_side**2 / n
        realistic_total += min_score
    print(f'Efficiency {efficiency*100:.0f}%: {realistic_total:.6f}')

print(f'\nCurrent best: {total:.6f}')
print(f'Target: 68.919154')

In [ ]:
# Key insight: The target (68.919) is achievable!
# Current efficiency is around 50-55%
# Target requires ~55-60% efficiency

# Let's see what the per-N efficiency looks like
print('Per-N efficiency analysis (sample):')
print('='*60)
for n in [1, 5, 10, 20, 50, 100, 150, 200]:
    actual_score = scores[n]
    actual_side = np.sqrt(actual_score * n)
    theoretical_side = np.sqrt(n * tree_area)
    efficiency = (theoretical_side / actual_side) ** 2
    print(f'N={n:3d}: score={actual_score:.4f}, side={actual_side:.4f}, efficiency={efficiency*100:.1f}%')

In [ ]:
# CRITICAL ANALYSIS: Where can we gain 1.711 points?

# Option 1: Improve ALL N values by 2.42%
# Option 2: Focus on specific N ranges

print('\nStrategic options to close 1.711 point gap:')
print('='*60)

# Option 1: Uniform improvement
print('\nOption 1: Uniform 2.42% improvement across all N')
print('  -> Requires improving EVERY N value by 2.42%')
print('  -> This is VERY difficult given solutions are already optimized')

# Option 2: Focus on large N (where most score comes from)
print('\nOption 2: Focus on large N (101-200)')
large_n_score = sum(scores[n] for n in range(101, 201))
print(f'  -> Large N contributes {large_n_score:.4f} ({large_n_score/total*100:.1f}%)')
print(f'  -> Need {1.711/large_n_score*100:.1f}% improvement in this range')

# Option 3: Find N values with poor efficiency
print('\nOption 3: Find N values with worst efficiency (most room for improvement)')
efficiencies = {}
for n in range(1, 201):
    actual_score = scores[n]
    actual_side = np.sqrt(actual_score * n)
    theoretical_side = np.sqrt(n * tree_area)
    efficiencies[n] = (theoretical_side / actual_side) ** 2

worst_efficiency = sorted(efficiencies.items(), key=lambda x: x[1])[:10]
print('Worst efficiency N values:')
for n, eff in worst_efficiency:
    print(f'  N={n:3d}: efficiency={eff*100:.1f}%, score={scores[n]:.4f}')

In [ ]:
# CONCLUSION: The target IS achievable but requires:
# 1. Finding better configurations for specific N values
# 2. The current solutions are at LOCAL optima, not GLOBAL optima
# 3. Need fundamentally different initial configurations

print('\n' + '='*60)
print('CONCLUSION')
print('='*60)
print(f'Current best: {total:.6f}')
print(f'Target: 68.919154')
print(f'Gap: {total - 68.919154:.6f} (2.42%)')
print()
print('The target IS achievable because:')
print('1. Current packing efficiency is ~50-55%')
print('2. Target requires ~55-60% efficiency')
print('3. This is within the range of good polygon packing')
print()
print('What we need:')
print('1. DIFFERENT initial configurations (not just SA on existing)')
print('2. Focus on N values with worst efficiency')
print('3. Try asymmetric solutions (per discussion with 34 votes)')
print('4. Consider tessellation patterns for large N')